## Part 1. Forgetful ChatBot

In [5]:
from pprint import pprint
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [6]:
# make sure OPENAI_API_KEY environment variable is defined
chatGPT = ChatOpenAI()

In [7]:
messages = [
    HumanMessage(content="What is the population of the German capital Berlin?")
]
print(chatGPT(messages))


content="According to the most recent data, the population of Berlin, Germany's capital, is approximately 3.8 million people." additional_kwargs={} example=False


In [8]:
messages = [
    HumanMessage(content="How does it compare to the populatio of London?")
]
print(chatGPT(messages))

content='To accurately compare the population of London to another location, we would need specific information about the population numbers of both locations. As of 2021, the estimated population of London is around 9 million people. Therefore, to compare it with another location, we would need the population figure of that specific place. Please provide the population of the location you want to compare with London, and I can help you make a comparison.' additional_kwargs={} example=False


## Part 2. + memory

In [21]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.save_context(
    {"input": "What is the population of the German capital Berlin?"}, 
    {"output": "As of 2021, the estimated population of Berlin, the capital city of Germany, is around 3.8 million."}
)

In [22]:
print(memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What is the population of the German capital Berlin?', additional_kwargs={}, example=False), AIMessage(content='As of 2021, the estimated population of Berlin, the capital city of Germany, is around 3.8 million.', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history'


In [9]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [10]:
# make sure OPENAI_API_KEY environment variable is defined
chatGPT = ChatOpenAI()

template = """Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat_history")
memory.save_context(
    {"input": "What is the population of the German capital Berlin?"}, 
    {"output": "As of 2021, the estimated population of Berlin, the capital city of Germany, is around 3.8 million."}
)

chat_with_memory = LLMChain(
    llm=chatGPT,
    prompt=prompt,
    verbose=False,
    memory=memory
)
pprint(chat_with_memory({"question":"How does it compare to the populatio of London?"}))

{'chat_history': 'Human: What is the population of the German capital Berlin?\n'
                 'AI: As of 2021, the estimated population of Berlin, the '
                 'capital city of Germany, is around 3.8 million.',
 'question': 'How does it compare to the populatio of London?',
 'text': 'As of 2021, the estimated population of London is around 8.9 '
         'million, making it significantly larger than Berlin.'}


In [12]:
pprint(chat_with_memory({"question":"How many languages are spoken in London?"}))

{'chat_history': 'Human: What is the population of the German capital Berlin?\n'
                 'AI: As of 2021, the estimated population of Berlin, the '
                 'capital city of Germany, is around 3.8 million.\n'
                 'Human: How does it compare to the populatio of London?\n'
                 'AI: As of 2021, the estimated population of London is around '
                 '8.9 million, making it significantly larger than Berlin.\n'
                 'Human: How many languages are spoken London?\n'
                 'AI: In London, over 300 languages are spoken, making it one '
                 'of the most linguistically diverse cities in the world.',
 'question': 'How many languages are spoken in London?',
 'text': 'In London, over 300 languages are spoken, making it one of the most '
         'linguistically diverse cities in the world.'}


## Part 3. +  knowledge

In [20]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Chain for having a conversation based on retrieved documents.
from langchain.chains  import ConversationalRetrievalChain
import langchain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings


info_about_berlin_loader = TextLoader("real-time-information-on-berlin.txt")

vectorstore = Chroma.from_documents(documents=info_about_berlin_loader.load(), 
                                    embedding=OpenAIEmbeddings())


chatGPT = ChatOpenAI()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_with_memory_and_knowledge = ConversationalRetrievalChain.from_llm(
    llm=chatGPT,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
    memory=memory,
    # verbose=True
)

In [17]:
pprint(chat_with_memory_and_knowledge({"question":"What is the population of the German capital Berlin?"}))

{'answer': 'As of 2023, the population of Berlin is 5,450,451.',
 'chat_history': [HumanMessage(content='What is the population of the German capital Berlin?', additional_kwargs={}, example=False),
                  AIMessage(content='As of 2023, the population of Berlin is 5,450,451.', additional_kwargs={}, example=False)],
 'question': 'What is the population of the German capital Berlin?'}


In [21]:
langchain.debug = True
pprint(chat_with_memory_and_knowledge({"question":"How many languages are spoken in London?"}))

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "How many languages are spoken in London?",
  "chat_history": []
}
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "How many languages are spoken in London?",
  "context": "As of 2023 Berlin population is 5450451. 51% are women."
}
[llm/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\nAs of 2023 Berlin population is 5450451. 51% are women.\nHuman: How many languages are 